In [ ]:
#imports 
import json
import collections
import more_itertools
import seaborn
import pandas as pd
import random
import subprocess
import pathlib
import numpy as np
import datetime
import tqdm
import csv
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
#nltk.download('stopwords')
#nltk.download('wordnet')
#nltk.download('omw-1.4')
import mmap
from tqdm import tqdm
import openpyxl
import more_itertools

In [ ]:
#data -- import/upload xlsx spreadsheet
data = pd.read_excel('/pscratch/sd/m/masare/sacct/incident_sla_AY23_complete.xlsx')
#data = pd.read_excel('/pscratch/sd/m/masare/sacct/incident_sla.xlsx')

In [ ]:
data.info()

In [ ]:
data.head()

In [ ]:
data.columns

In [ ]:
#Rename Colum 'Number' to 'ID' into the dataframe
data = data.rename(columns={"Number":"ID"})


In [ ]:
data.columns # RECHECK COLUMNS

In [ ]:
type(data)
data.dtypes

In [ ]:
data['ID']=data['ID'].astype(str) #ensuring ID column is of string type
data['ID']=data['ID'].str[3:] #removing first three characters
data['ID']=pd.to_numeric(data['ID'],errors='coerce') #converting ID column into numberic
data.head()

In [ ]:
#selecting desired columns
columns_to_keep = ['ID',
                 'Created',  
                'Title',
                'Additional comments',
                'Close notes',
                'Comments and Work notes',
                'Staff work notes (NERSC private)']
data = data.loc[:, columns_to_keep].copy()

In [ ]:
#removing spaces from column names
data.columns = data.columns.str.replace(' ', '')

In [ ]:
data.loc[:, 'Created']= pd.to_datetime(data['Created'], errors = 'coerce')

In [ ]:
print(data.dtypes)

In [ ]:
data.columns

In [ ]:
#convert columns from object to str. I have to use ".loc" to avoid the copy warning
object_columns =[ 'Title', 
                 'Additionalcomments', 
                 'Closenotes',
                 'CommentsandWorknotes', 
                 'Staffworknotes(NERSCprivate)']
for col in object_columns:
    data.loc[:, col] = data.loc[:, col].astype(str)

In [ ]:
#clean 
   #isolate import columns, drop unimportant columns
   #remove time stamps (can look at my code and use that) 


text_cols = [ 'Created', 
             'Title',
            'Additionalcomments',
             'Closenotes',
            'CommentsandWorknotes',
           'Staffworknotes(NERSCprivate)'
            ]

data = data[['ID'] + text_cols].copy()
data.fillna('', inplace=True)
data = data.astype(str)

## EP -- Let's use this cell to name the columns we want our data frame. This is a little different than what I did in
# 'ticket_text_mine.py`


In [ ]:
data[text_cols[1]]

In [ ]:
###### DATA CLEANING AND ISOLATION

In [ ]:
# :-----------------------------------------------------------------------------
#clean 
   #isolate import columns, drop unimportant columns
   #remove time stamps (can look at my code and use that) 
# Function: process_comment_text
# Input: ind -- index from the pandas dataframe that holds the cleaned xlsx data
# Output: Single row that includes, index, number, and text columns combined and
# #wed jan 18 2023 812 pm 201128 - shane canon (staff work note (nersc private)) 

 


def process_comment_text(text):
    text = re.sub(r'On \w+, w+ \d+, d{4} at \d{2}:\d{2} \w{2} \d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2} - \w+ \w+ \(Staff work notes \(NERSC private\)\)','',text)
    text = re.sub(r'\w+, w+ \d+, d{4} at \d{2}:\d{2} \w{2} \d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2} - \w+ \w+ \(Staff work notes \(NERSC private\)\)','',text)
    text = re.sub(r'\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2} - \w+ \w+ \(Additional comments\)', '', text)
    text = re.sub(r'\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2} - \w+-\w+ \w+ \(Additional comments\)', '', text)
#the new ones above seems not to work


    # Removing timestamps
    #text = re.sub(r'\b\w{3}\s+\w{3}\s+\d{1,2}\s+\d{4}\s+\d{1,2}:\d{2}\s+\w{2}\b', '', text)
    #text = re.sub(r'\b\w{3}\s+\w{3}\s+\d{1,2}\s+\d{4}\s+\d{1,2}\d{2}\s+\w{2}\b', '', text, flags=re.IGNORECASE)
    #text = re.sub(r'\b(?:mon|tue|wed|thu|fri|sat|sun)\s+(?:jan|feb|mar|apr|may|jun|jul|aug|sep|oct|nov|dec)\s+\d{1,2}\s+\d{4}\s+\d{1,2}[:]\d{2}\s+\w{2}\b', '', text, flags=re.IGNORECASE)
    text = re.sub(r'\b\w{3}\s+\w{3}\s+\d{1,2}\s+\d{4}\s+\d{1,2}\d{2}\s+\w{2}\b', '', text) 
    #text = re.sub(r'\b\(?:sat|sun|mon|tue|wed|thu|fri)\.?(?:\w*day)?\s?\b', '',text)
    #mon[a-z\.]*\s+|tue[a-z\.]*\s+|wed[a-z\.]*\s+|thu[a-z\.]*\s+|fri[a-z\.]*\s+|sat[a-z\.]*\s+|sun[a-z\.]*\s+

    
    # Removing bracketed content
    text = re.sub(r'\[.*?\]', '', text)
    text = re.sub(r'\(.*?\)', '', text)
    # Removing extra whitespace
    text = ' '.join(text.split())

    #Removing names
    #text = re.sub(r'\b[a-zA-Z][a-z]*\b(?:\s+[a-zA-Z][a-z]*)*', '', text) 
    text = re.sub(r'\b[a-zA-Z-]+$\b','', text)
    #phone numbers
    text = re.sub(r'\d\d\d-\d\d\d-\d\d\d\d','', text)
  
    
    text = re.sub(r'\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2} - Guest \(Additional comments\)', '', text)
    text  = re.sub(r'\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2} - System \(Additional comments\)', '', text)
    text = re.sub(r'\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2} - \w+ \w+ \(Additional comments\)', '', text)
    text = re.sub(r'\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2} - \w+ \w+ \(Staff work notes \(NERSC private\)\)', '', text)
    text = re.sub(r'\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2} - Guest \(Staff work notes \(NERSC private\)\)', '', text)
    text = re.sub(r'@\w+ \w+', '', text)
    text = re.sub(r'\d{4}-\d{2}-\d{2}', '', text)
    # text_data = re.sub(r'\d{2}:\d{2}:\d{2}', '', text) #remove system times
    # text_data = re.sub(r'\(Additional comments\)', '', text)
    # text_data = re.sub(r'\(Staff work notes \(NERSC private\)\)', '', text)
   
    # Convert text to lowercase
    text = text.lower()
    # Remove HTML links
    text = re.sub(r'https?:\/\/(www\.)?[-a-zA-Z0-9@:%._\+~#=]{1,256}\.[a-zA-Z0-9()]{1,6}\b([-a-zA-Z0-9()@:%_\+.~#?&//=]*)', '', text)
    
    # Remove punctuations
    # text = re.sub(r'\.','', text)
    text = re.sub(r',','', text)
    text = re.sub(r':','', text)
    text = re.sub(r'\?','', text)
    
    # Remove specific words like 'hi', 'hello'
    unwanted_words = ['hi', 'hello','email','organization','laboratory', 'state','bye','mr','thanks','day']
    pattern = r'\b(?:{})\b'.format('|'.join(unwanted_words))
    text = re.sub(pattern, '', text)
    
    #remove phonenumber and emails
    phone_pattern = r'(\+?\d{1,2}[-.\s]?)?(\(?\d{3}\)?[-.\s]?)?\d{3}[-.\s]?\d{4}'
    text = re.sub(phone_pattern, '', text)
    
    email_pattern = r'[\w]+@[\w]+.\w{3}'
    text = re.sub(email_pattern, '', text)
        
        
    # removing quotes can be problematic
    # text = re.sub(r'\"','', text)
    # text = re.sub(r'\'','', text)
    text = text.split()
    wl = WordNetLemmatizer()
    text = [wl.lemmatize(word) for word in text if not word in set(stopwords.words('english'))]
    text = ' '.join(text)
    return text

In [ ]:
%%time
for col in text_cols: 
    if col not in data.columns: 
        raise ValueError(f"Column '{col}' not found in DataFrame") 


data[text_cols] = data[text_cols].fillna('').astype(str)
# Concatenate the text columns
data['text_data'] = data[text_cols].apply(lambda row: ' '.join(row.values.astype(str)), axis=1)
# Apply the cleaning function
data['text_data'] = data['text_data'].apply(process_comment_text)

In [ ]:
cleaned_data = data[['ID','Created','text_data']]  

In [ ]:
cleaned_data.head()

In [ ]:
cleaned_data.to_csv('/global/u2/m/masare/ticket-text-analysis/ticket_cleaned.csv', index=False)
#data_df = pd.read_csv('/global/u2/m/masare/ticket-text-analysis/Ticket.csv')

In [ ]:
cleaned_data['text_data'].iloc[0]

In [ ]:
import spacy

In [ ]:
nlp = spacy.load("en_core_web_lg")

In [ ]:
 def clean_text(text):
    if pd.isna(text):
        return text
    
    # Process text with spaCy
    doc = nlp(text)
    
    # Remove named entities
    for ent in doc.ents:
        if ent.label_ in ["PERSON", "ORG", "GPE"]:  # Consider adding other relevant types if needed
            text = text.replace(ent.text, '')

In [ ]:
 data['cleaned_text'] = data['text_data'].apply(clean_text)

In [ ]:
print(data[['cleaned_text']])

In [ ]:
cleaned_data['text_data'].iloc[0]

In [ ]:
my_text = cleaned_data['text_data']

In [ ]:
my_text

In [ ]:
len(my_text)

In [ ]:
nlp.pipeline

In [ ]:
my_text[0]

In [ ]:
my_text.head

In [ ]:
def remove_entity_names(text):
  doc = nlp(text)
  new_text = []
  for token in doc:
    if token.ent_type_ == "":
      new_text.append(token.text)
    else:
      new_text.append(" ")
  return " ".join(new_text)


In [ ]:
cleaned_data['text_column_no_entities'] = cleaned_data['text_data'].apply(remove_entity_names)



#cleaned_data['text_data'] = cleaned_data['text_data'].apply(unicode)

In [ ]:
print(cleaned_data.head)

In [ ]:
cleaned_data['text_column_no_entities'][2]

In [ ]:
doc1 =  cleaned_data['text_column_no_entities']

In [ ]:
#from spacy import displacy
#displacy.render(doc1, style='ent', jupyter=True)

In [ ]:
for doc1 in cleaned_data['text_column_no_entities']:
  doc1 = nlp(doc1)
  displacy.render(doc1, style='ent', jupyter=True)


In [ ]:
def remove_dates(text):
  doc = nlp(text)
  new_text = []
  for token in doc:
    if token.ent_type_ == "DATE":
      new_text.append(" ")
    else:
      new_text.append(token.text)
  return " ".join(new_text)

cleaned_data['text_column_no_dates'] = cleaned_data['text_column_no_entities'].apply(remove_dates)

for doc1 in cleaned_data['text_column_no_dates']:
  doc1 = nlp(doc1)
  displacy.render(doc1, style='ent', jupyter=True)


In [ ]:
from spacy import displacy
displacy.render(doc1, style='dep', jupyter=True)